In [5]:
# Standard imports
import numpy as np
import pandas as pd
import ast 

# For scraping
import requests
from bs4 import BeautifulSoup

# For adding delays so that we don't spam requests
import time

import string

import re
import json


In [6]:

def get_genre(str_dic):
    genre_list=[]
    for i in str_dic:
        if i == '"genre":':
            j=str_dic.index(i)+1
            while str_dic[j] not in ['],','"actor":','"director":','"creator":','"datePublished":','}</script>]']:
                for punctuation_mark in string.punctuation:
                    str_dic[j]=str_dic[j].replace(punctuation_mark,'')
                if str_dic[j]!="":   
                    genre_list.append(str_dic[j])
                j+=1
            break  

    return genre_list

In [7]:
df_twothousandstwenties=pd.read_csv('/Users/SanaKrichen/Desktop/Brainstation/Capstone Project/twothousandstwenties/df_twothousandstwenties.csv')
df_twothousandstwenties

,tconst,primaryTitle,startYear
0,tt0062336,El Tango del Viudo y Su Espejo Deformante,2020
1,tt0120589,Arigo,2020
2,tt10006160,Kala azar,2020
3,tt10032266,Asia,2020
4,tt10039354,LasseMajas detektivbyrå - Tågrånarens hemlighet,2020
...,...,...,...
1234,tt9894440,Min pappa Marianne,2020
1235,tt9897258,Rosa pietra stella,2020
1236,tt9902160,Herself,2020
1237,tt9910530,Svartklubb,2020


In [8]:
scrap_df = pd.DataFrame(columns=['tconst', 'genre', 'plot'])
i=0

for tconst in df_twothousandstwenties['tconst']:


    imdb_to_scrap = {'tconst':"", 'genre':[], 'plot':""}
    
    response2=requests.get(f'https://www.imdb.com/title/{tconst}/plotsummary')
    soup2 = BeautifulSoup(response2.content,'html.parser')
        
    if soup2.title.text.strip() != '404 Error - IMDb':
        plot=soup2.find('ul', id="plot-synopsis-content").text.strip()


        if plot!='It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.':

            imdb_to_scrap = {'tconst':tconst, 'genre':[], 'plot':plot}

            # Send a get request and assign the response to a variable
            response1 = requests.get(f'https://www.imdb.com/title/{tconst}/')
            soup1 = BeautifulSoup(response1.content,'html.parser')


            str_dic=str(soup1.findAll('script', {'type':'application/ld+json'})).split()
            imdb_to_scrap['genre']=[get_genre(str_dic)]


            df = pd.DataFrame.from_dict(imdb_to_scrap)
            scrap_df = scrap_df.append(df, ignore_index=True)

            #follow progress: print tconst every 50 iteration
            i+=1
            if i%50==0:
                print(f'after 50 append {tconst}')


scrap_df


after 50 append tt9214832


,tconst,genre,plot
0,tt10044182,"[Action, Horror, Thriller]",Several news reports that an unknown viral epi...
1,tt10122486,"[Comedy, Horror]",I Need You Dead! follows the story of a young ...
2,tt1051906,"[Horror, Mystery, SciFi, Thriller]",Trapped in an abusive relationship with her fi...
3,tt10608770,"[Crime, Thriller]",One night four friends went to a nightclub for...
4,tt10715268,[Documentary],In MISTER SOUL - A STORY ABOUT DONNY HATHAWAY ...
5,tt11239374,[Thriller],Scumbag\nThou shalt not steal. Thou shalt not ...
6,tt11312640,[Documentary],Tenere is about the true story of African peop...
7,tt11364376,[Documentary],"A green space in the middle of a metropolis, t..."
8,tt11369498,[Drama],In a rural Bengali village soaked with traditi...
9,tt11390102,[Documentary],Vivos is a documentary feature film by artist ...


In [10]:
scrap_df

,tconst,genre,plot
0,tt10044182,"[Action, Horror, Thriller]",Several news reports that an unknown viral epi...
1,tt10122486,"[Comedy, Horror]",I Need You Dead! follows the story of a young ...
2,tt1051906,"[Horror, Mystery, SciFi, Thriller]",Trapped in an abusive relationship with her fi...
3,tt10608770,"[Crime, Thriller]",One night four friends went to a nightclub for...
4,tt10715268,[Documentary],In MISTER SOUL - A STORY ABOUT DONNY HATHAWAY ...
5,tt11239374,[Thriller],Scumbag\nThou shalt not steal. Thou shalt not ...
6,tt11312640,[Documentary],Tenere is about the true story of African peop...
7,tt11364376,[Documentary],"A green space in the middle of a metropolis, t..."
8,tt11369498,[Drama],In a rural Bengali village soaked with traditi...
9,tt11390102,[Documentary],Vivos is a documentary feature film by artist ...


In [9]:
scrap_df.to_csv (r'/Users/SanaKrichen/Desktop/Brainstation/Capstone Project/twothousandstwenties/scraped_df_twothousandstwenties.csv', index = False, header=True)